In [1]:
import sys; sys.path.append('../3rdparty/ElasticRods/python')
import elastic_rods, elastic_knots
import numpy as np, matplotlib.pyplot as plt, time, io, os

from helpers import *
from parametric_curves import *
import py_newton_optimizer

from linkage_vis import LinkageViewer as Viewer, CenterlineViewer
from tri_mesh_viewer import PointCloudViewer, PointCloudMesh

%load_ext autoreload
%autoreload 2

import parallelism
parallelism.set_max_num_tbb_threads(1)

In [6]:
# Load the centerline from file...
file = '../data/NoCollision/0001.obj'
file2 = '../data/NoCollision/0033.obj'
file = '../data/NoCollision/reduced0001.obj'
file2 = '../data/NoCollision/reduced0033.obj'
rod_radius = 0.2
material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
A = read_nodes_from_file(file2)
B = read_nodes_from_file(file)
pr = define_periodic_rod(A, material)
rod_list = elastic_knots.PeriodicRodList([pr])
nPoints = len(A)
print(B.shape)
problemOptions = elastic_knots.ContactProblemOptions()
problemOptions.hasCollisions = True
problemOptions.contactStiffness = 10000
problemOptions.dHat = 2*rod_radius

contactProblem = elastic_knots.ContactProblem(rod_list,problemOptions)

(100, 3)


In [7]:
view = Viewer(rod_list, width=1024, height=800)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.28, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0),…

In [8]:
def moveTo(p, target):
    k = 2
    F = k*(target - p)
    return np.array(F)

def getSpringForce(R, R_next, k=2.0):
    d = R_next - R
    F = k*d 
    return F

def getMaxDist(points):
    diffs = points[1:] - points[:-1] 
    dists = np.linalg.norm(diffs, axis=1)   
    return np.max(dists)

def callback(problem, iteration):
    if iteration % 5 == 0:
        view.update()

In [5]:

#start pulling toward B
target=B
print(f"initial Max dist: {getMaxDist(A)}")
for n in range(nPoints+1):
    
    for it in range(1000):
        DoFs = contactProblem.getDoFs().copy()
        points = DoFs[:nPoints*3].reshape((-1, 3))
        grad = contactProblem.contactForces()
        contactForce = grad[:nPoints*3].reshape((-1, 3))
        
        for i in range(0, n):
            if i == nPoints-1:
                springForce = getSpringForce(points[i], points[0]) + getSpringForce(points[i], points[i-1])
            else:
                springForce = getSpringForce(points[i], points[i+1])+ getSpringForce(points[i], points[i-1])
                
            F = 0.001*springForce+0.0001*moveTo(points[i], target[i]) +contactForce[i]
            F = F/ np.linalg.norm(F)
            points[i] = (points[i] +0.01*F)
            
            
        for i in range(n,nPoints):
            
            if i == nPoints-1:
                springForce = getSpringForce(points[i], points[0])+ getSpringForce(points[i], points[i-1])
            else:
                springForce = getSpringForce(points[i], points[i+1])+ getSpringForce(points[i], points[i-1])
            F = 0.001*springForce+contactForce[i]
            F = F/ np.linalg.norm(F)
            points[i] = (points[i] +0.01*F)
        
            
           
        DoFs[:nPoints*3]=points.flatten()
        contactProblem.setDoFs(DoFs.flatten())
        view.update()
        if it % 100 == 0 :
            print(f"{n*1000 + it} Max dist: {getMaxDist(points)}")

initial Max dist: 7.146963455309995
0 Max dist: 7.13539288301482
100 Max dist: 6.7219598836689105
200 Max dist: 6.3657108627036285
300 Max dist: 6.093851066883221
400 Max dist: 5.828575750940895
500 Max dist: 5.584264720881001
600 Max dist: 5.352947475435829
700 Max dist: 5.121838824463223
800 Max dist: 4.898059517445305
900 Max dist: 4.687060916105735
1000 Max dist: 4.483992120222613
1100 Max dist: 4.29435604793797
1200 Max dist: 4.119756751054133
1300 Max dist: 4.009059736125777
1400 Max dist: 3.9414911501954983
1500 Max dist: 3.8749861785209716
1600 Max dist: 3.8080387906408673
1700 Max dist: 3.737610758455669
1800 Max dist: 3.6654122458296587
1900 Max dist: 3.587568078211243
2000 Max dist: 3.5149221207580346
2100 Max dist: 3.4366401016860206
2200 Max dist: 3.357464270537616
2300 Max dist: 3.277099120965455
2400 Max dist: 3.1959975480383642
2500 Max dist: 3.117042048742393
2600 Max dist: 3.0374486923925272
2700 Max dist: 2.954881561438176
2800 Max dist: 2.877080673390445
2900 Max di

In [66]:
from helpers import write_obj
file = '../data/test.obj'
write_obj(file, rod_list)
rod_radius = 0.2
material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
A = read_nodes_from_file(file)
pr = define_periodic_rod(A, material)
rod_list = elastic_knots.PeriodicRodList([pr])
view = Viewer(rod_list, width=1024, height=800)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.28, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0),…

In [67]:

def callback(problem, iteration):
    if iteration % 5 == 0:
        view.update()

problemOptions = elastic_knots.ContactProblemOptions()
problemOptions.contactStiffness = 1e3
problemOptions.dHat = 2*rod_radius

fixedVars = []   # all the degrees of freedom can be modified by the optimizer
optimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
optimizerOptions.niter = 10000
optimizerOptions.gradTol = 1e-8
hessianShift = 1e-4 * compute_min_eigenval_straight_rod(pr)

report = elastic_knots.compute_equilibrium(
rod_list, problemOptions, optimizerOptions, 
externalForces=np.zeros(rod_list.numDoF()),
softConstraints=[],
callback=callback,
hessianShift=hessianShift
)
view.update()

0	44.7346	56.8535	56.8535	1	1
1	43.6469	54.1329	54.1329	1	1
2	41.7897	49.628	49.628	1	1
3	40.951	47.6337	47.6337	1	1
4	39.466	44.2293	44.2293	1	1
5	38.7633	42.6964	42.6964	1	1
6	36.8355	39.8545	39.8545	1	1
7	35.3212	38.3692	38.3692	1	1
8	26.2719	31.4846	31.4846	1	1
9	15.0365	8.64152	8.64152	1	1
10	14.6032	7.62867	7.62867	1	1
11	14.4129	7.29188	7.29188	1	1
12	13.9706	6.81265	6.81265	0.5	1
13	12.3226	4.88508	4.88508	1	1
14	11.5686	3.72284	3.72284	1	1
15	10.8567	3.08596	3.08596	1	1
16	10.609	2.8966	2.8966	1	1
17	10.2803	2.56343	2.56343	1	1
18	10.1578	2.43941	2.43941	1	1
19	9.97561	2.22727	2.22727	1	1
20	9.90004	2.13636	2.13636	1	1
21	9.77671	1.98138	1.98138	1	1
22	9.72204	1.91805	1.91805	1	1
23	9.62562	1.81655	1.81655	1	1
24	9.58137	1.76348	1.76348	1	1
25	9.49914	1.69019	1.69019	1	1
26	9.4586	1.65183	1.65183	1	1
27	9.36291	1.59472	1.59472	0.015625	1
28	9.07417	1.71389	1.71389	1	1
29	8.65646	1.61682	1.61682	1	1
30	8.3452	2.21877	2.21877	1	1
31	7.9923	2.50583	2.50583	0.5	1
32	7.81452	7.5252